In [1]:
!nvidia-smi


'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [4]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled accelerate-1.6.0
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached accelerate-1.6.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached accelerate-1.6.0-py3-none-any.whl (354 kB)

   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ------

In [5]:
!pip install evaluate


In [8]:
!pip install torch --upgrade

  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 1.6/212.5 MB 8.3 MB/s eta 0:00:26
    --------------------------------------- 3.7/212.5 MB 8.7 MB/s eta 0:00:24
   - -------------------------------------- 5.5/212.5 MB 9.1 MB/s eta 0:00:23
   - -------------------------------------- 7.3/212.5 MB 9.1 MB/s eta 0:00:23
   - -------------------------------------- 9.2/212.5 MB 8.9 MB/s eta 0:00:23
   - -------------------------------------- 10.5/212.5 MB 8.4 MB/s eta 0:00:25
   -- ------------------------------------- 12.3/212.5 MB 8.4 MB/s eta 0:00:24
   -- ------------------------------------- 14.4/212.5 MB 8.5 MB/s eta 0:00:24
   --- ------------------------------------ 16.0/212.5 MB 8.5 MB/s eta 0:00:24
   --- ------------------------------------ 17.8/212.5 MB 8.5 MB/s eta 0:00:23
   --- ------------------------------------ 19.7/212.5 MB 8.5 MB/s et

  You can safely remove it manually.


In [11]:
!pip install torch==2.1.2 torchvision torchaudio


  Using cached torch-2.1.2-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Using cached torchvision-0.22.0-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached torchaudio-2.7.0-cp311-cp311-win_amd64.whl.metadata (6.7 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.21.0-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached torchvision-0.20.1-cp311-cp311-win_amd64.whl.metadata (6.2 kB)
  Using cached torchvision-0.20.0-cp311-cp311-win_amd64.whl.metadata (6.2 kB)
  Using cached torchvision-0.19.1-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
  Using cached torchvision-0.19.0-1-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
  Using cached torchvision-0.18.1-cp311-cp311-win_amd64.whl.metadata (6.6 kB)
  Using cached torchvision-0.18.0-cp311-cp311-win_amd64.whl.metadata (6.6 kB)
INFO: pip is still looking at multiple versions of torchvision to determine whic

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\gauth\\anaconda3\\envs\\hfacetexts\\lib\\site-packages\\functorch\\'



INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.5/192.3 MB 5.6 MB/s eta 0:00:35
   ---------------------------------------- 1.0/192.3 MB 3.6 MB/s eta 0:00:54
   ---------------------------------------- 2.1/192.3 MB 3.3 MB/s eta 0:00:59
    --------------------------------------- 3.1/192.3 MB 3.4 MB/s eta 0:00:57
    -----------

In [15]:
!pip install torch==2.1.2 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu



Looking in indexes: https://download.pytorch.org/whl/cpu


In [4]:
import torch
print(torch.__version__)

2.1.2+cpu


In [3]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
from evaluate import load as load_metric

import pandas as pd


from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download('punkt')


c:\Users\gauth\anaconda3\envs\HfaceTextS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gauth\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_ckpt = 'google/pegasus-cnn_dailymail'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

c:\Users\gauth\anaconda3\envs\HfaceTextS\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gauth\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\gauth\anaconda3\envs\HfaceTextS\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedS

In [6]:
device

'cpu'

In [7]:
# download and unzip data

!wget https://github.com/dystinktbeatz/Text-summarizer-project/raw/refs/heads/main/samsum_dataset_G.zip
!unzip summarizer-data.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
import urllib.request
import zipfile
import os

# Download the file
url = "https://github.com/dystinktbeatz/Text-summarizer-project/raw/refs/heads/main/samsum_dataset_G.zip"
local_zip_path = "samsum_dataset_G.zip"

urllib.request.urlretrieve(url, local_zip_path)
print("✅ Downloaded successfully!")

# Extract the zip file
with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
    zip_ref.extractall("extracted_data")  # choose your target folder

print("✅ Unzipped successfully!")


✅ Downloaded successfully!
✅ Unzipped successfully!


In [13]:

print(os.listdir())


['01_data_ingestion.ipynb', 'extracted_data', 'samsum_dataset_G.zip', 'Text_Summarization.ipynb', 'trials.ipynb']


In [15]:
import os
print(os.listdir("extracted_data"))


['samsum-test.csv', 'samsum-train.csv', 'samsum-validation.csv', 'samsum_dataset']


In [17]:


path = "extracted_data/samsum_dataset"
print("Is it a file?", os.path.isfile(path))
print("Is it a directory?", os.path.isdir(path))
print("Contents (if directory):", os.listdir(path) if os.path.isdir(path) else "N/A")


Is it a file? False
Is it a directory? True
Contents (if directory): ['dataset_dict.json', 'test', 'train', 'validation']


In [18]:
from datasets import load_from_disk
dataset_samsum = load_from_disk("extracted_data/samsum_dataset")
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [21]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f'Split lengths: {split_lengths}')
print(f'Features: {dataset_samsum["train"].column_names}')
print("\nDialogue:")

print(dataset_samsum['test'][1]['dialogue'])

print("\nSummary:")
print(dataset_samsum['test'][1]['summary'])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [22]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [23]:
dataset_samsun_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\gauth\anaconda3\envs\HfaceTextS\Lib\site-packages\transformers\tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:00<00:00, 1017.37 examples/s]


In [24]:
#  Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [25]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    #evaluation_strategy='epoch',
    #logging_delay=500,
    save_steps=1e6,
    gradient_accumulation_steps=16,
    #no_deprecation_warning=True,
    report_to="none"
    #save_total_limit=1
)

In [26]:
# Define a Trainer instance
trainer = Trainer(
    model=model_pegasus,
    args=trainer_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_samsun_pt['test'], #!!! always use 'train' here. test is used because the size is small here
    eval_dataset=dataset_samsun_pt['validation']


)


C:\Users\gauth\AppData\Local\Temp\ipykernel_19356\1708947673.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [27]:
trainer.train()

Step,Training Loss
10,3.110100
20,3.046700
30,3.161300
40,2.990100


: 

In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device),
                                          length_penalty=0.8, num_beams=8, max_length=128)
        '''parameter for length penalty ensures that the model does not generate sequences that are too long.'''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the reference to the metric.

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

        #Finally compute and return the ROUGE scores
        score = metric.compute()
    return score

In [ ]:
rouge_names = ["rouge1","rouge2","rougeL","rougeeLsum"]
rouge_metric  = load_metric('rouge')

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10],
    rouge_metric,
    trainer.model,
    tokenizer,
    batch_size = 2,
    column_text = 'dialogue',
    column_summary= 'summary')
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

In [ ]:
# Save model

model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
# Save tokenizer
tokenizer.save_pretrained("tokenizer")

In [ ]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

print("Dialogue:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])